<a href="https://colab.research.google.com/github/mushhub/my-first-blockchain/blob/main/ERC20_multi_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Google Colaboratory用ERC20トークン一括送金プログラム

# 必要なライブラリのインストール
!pip install web3

import time
from web3 import Web3
import json
import pandas as pd
from google.colab import files

# ERC20トークンのABI（Application Binary Interface）
ERC20_ABI = [
    {
        "constant": True,
        "inputs": [],
        "name": "name",
        "outputs": [{"name": "", "type": "string"}],
        "payable": False,
        "stateMutability": "view",
        "type": "function"
    },
    {
        "constant": False,
        "inputs": [{"name": "_spender", "type": "address"}, {"name": "_value", "type": "uint256"}],
        "name": "approve",
        "outputs": [{"name": "", "type": "bool"}],
        "payable": False,
        "stateMutability": "nonpayable",
        "type": "function"
    },
    {
        "constant": True,
        "inputs": [],
        "name": "totalSupply",
        "outputs": [{"name": "", "type": "uint256"}],
        "payable": False,
        "stateMutability": "view",
        "type": "function"
    },
    {
        "constant": False,
        "inputs": [{"name": "_from", "type": "address"}, {"name": "_to", "type": "address"}, {"name": "_value", "type": "uint256"}],
        "name": "transferFrom",
        "outputs": [{"name": "", "type": "bool"}],
        "payable": False,
        "stateMutability": "nonpayable",
        "type": "function"
    },
    {
        "constant": True,
        "inputs": [],
        "name": "decimals",
        "outputs": [{"name": "", "type": "uint8"}],
        "payable": False,
        "stateMutability": "view",
        "type": "function"
    },
    {
        "constant": True,
        "inputs": [{"name": "_owner", "type": "address"}],
        "name": "balanceOf",
        "outputs": [{"name": "balance", "type": "uint256"}],
        "payable": False,
        "stateMutability": "view",
        "type": "function"
    },
    {
        "constant": True,
        "inputs": [],
        "name": "symbol",
        "outputs": [{"name": "", "type": "string"}],
        "payable": False,
        "stateMutability": "view",
        "type": "function"
    },
    {
        "constant": False,
        "inputs": [{"name": "_to", "type": "address"}, {"name": "_value", "type": "uint256"}],
        "name": "transfer",
        "outputs": [{"name": "", "type": "bool"}],
        "payable": False,
        "stateMutability": "nonpayable",
        "type": "function"
    },
    {
        "constant": True,
        "inputs": [{"name": "_owner", "type": "address"}, {"name": "_spender", "type": "address"}],
        "name": "allowance",
        "outputs": [{"name": "", "type": "uint256"}],
        "payable": False,
        "stateMutability": "view",
        "type": "function"
    }
]

class ERC20BulkTransfer:
    def __init__(self, provider_url, token_address, private_key):
        """
        初期化関数

        Args:
            provider_url (str): イーサリアムノードのURL (Infura, AlchemyなどのエンドポイントURL)
            token_address (str): ERC20トークンのコントラクトアドレス
            private_key (str): 送金元のプライベートキー (0xなしの形式)
        """
        self.web3 = Web3(Web3.HTTPProvider(provider_url))
        self.token_address = Web3.to_checksum_address(token_address)
        self.token_contract = self.web3.eth.contract(address=self.token_address, abi=ERC20_ABI)
        self.private_key = private_key
        self.account = self.web3.eth.account.from_key(private_key)
        self.sender_address = self.account.address

        # トークン情報を取得
        self.token_name = self.token_contract.functions.name().call()
        self.token_symbol = self.token_contract.functions.symbol().call()
        self.token_decimals = self.token_contract.functions.decimals().call()

        print(f"接続状況: {self.web3.is_connected()}")
        print(f"トークン名: {self.token_name}")
        print(f"トークンシンボル: {self.token_symbol}")
        print(f"トークンデシマル: {self.token_decimals}")
        print(f"送金元アドレス: {self.sender_address}")

        # 残高確認
        self.check_balance()

    def check_balance(self):
        """送金元アドレスの残高を確認"""
        balance = self.token_contract.functions.balanceOf(self.sender_address).call()
        eth_balance = self.web3.eth.get_balance(self.sender_address)

        # 読みやすい形式に変換
        readable_balance = balance / (10 ** self.token_decimals)
        readable_eth = self.web3.from_wei(eth_balance, 'ether')

        print(f"{self.token_symbol}残高: {readable_balance} {self.token_symbol}")
        print(f"ETH残高: {readable_eth} ETH")

        return readable_balance, readable_eth

    def upload_csv(self):
        """CSVファイルをアップロードして受取人リストを取得"""
        print("CSVファイルをアップロードしてください。フォーマット: address,amount")
        uploaded = files.upload()

        for filename in uploaded.keys():
            # CSVファイルを読み込む
            recipients_df = pd.read_csv(filename)
            return recipients_df

    def validate_addresses(self, addresses):
        """アドレスの有効性を検証"""
        valid_addresses = []
        invalid_addresses = []

        for addr in addresses:
            if self.web3.is_address(addr):
                valid_addresses.append(Web3.to_checksum_address(addr))
            else:
                invalid_addresses.append(addr)

        if invalid_addresses:
            print(f"無効なアドレスが見つかりました: {invalid_addresses}")

        return valid_addresses, invalid_addresses

    def send_tokens(self, recipients_df):
        """複数アドレスにトークンを送金"""
        # アドレスを検証
        addresses = recipients_df['address'].tolist()
        valid_addresses, invalid_addresses = self.validate_addresses(addresses)

        if invalid_addresses:
            print("無効なアドレスがあるため送金を中止します。")
            return

        # 有効なアドレスのみでデータフレームをフィルタリング
        valid_df = recipients_df[recipients_df['address'].apply(lambda x: self.web3.is_address(x))]

        # 合計送金額を計算
        total_amount = sum(valid_df['amount'])
        print(f"合計送金額: {total_amount} {self.token_symbol}")

        # 残高を確認
        balance, eth_balance = self.check_balance()
        if balance < total_amount:
            print(f"残高不足です。必要額: {total_amount} {self.token_symbol}, 保有額: {balance} {self.token_symbol}")
            return

        if eth_balance < 0.01:
            print(f"ETH残高が少なすぎます。トランザクション手数料に十分なETHが必要です。")
            return

        # ユーザー確認
        confirm = input(f"{len(valid_df)}件のアドレスに合計{total_amount} {self.token_symbol}を送金します。続行しますか？ (y/n): ")
        if confirm.lower() != 'y':
            print("送金をキャンセルしました。")
            return

        # 各アドレスに送金
        results = []
        nonce = self.web3.eth.get_transaction_count(self.sender_address)

        for index, row in valid_df.iterrows():
            address = Web3.to_checksum_address(row['address'])
            amount = row['amount']
            amount_in_wei = int(amount * (10 ** self.token_decimals))

            try:
                print(f"送金処理中... アドレス: {address}, 金額: {amount} {self.token_symbol}")

                # トランザクションを作成
                tx = self.token_contract.functions.transfer(
                    address,
                    amount_in_wei
                ).build_transaction({
                    'chainId': self.web3.eth.chain_id,
                    'gas': 100000,
                    'gasPrice': self.web3.eth.gas_price,
                    'nonce': nonce,
                })

                # トランザクションに署名
                signed_tx = self.web3.eth.account.sign_transaction(tx, self.private_key)

                # トランザクションを送信
                tx_hash = self.web3.eth.send_raw_transaction(signed_tx.rawTransaction)

                # レシートを待機
                receipt = self.web3.eth.wait_for_transaction_receipt(tx_hash)

                # 結果を追加
                status = "成功" if receipt.status == 1 else "失敗"
                results.append({
                    'address': address,
                    'amount': amount,
                    'tx_hash': tx_hash.hex(),
                    'status': status
                })

                print(f"トランザクション完了: {tx_hash.hex()}, ステータス: {status}")

                # nonceを増やす
                nonce += 1

                # レート制限対策のため少し待機
                time.sleep(1)

            except Exception as e:
                print(f"エラーが発生しました: {e}")
                results.append({
                    'address': address,
                    'amount': amount,
                    'tx_hash': None,
                    'status': f"エラー: {str(e)}"
                })

        # 結果をデータフレームに変換
        results_df = pd.DataFrame(results)
        print("\n送金結果:")
        print(results_df)

        # CSVファイルとして結果を保存
        results_filename = f"transfer_results_{int(time.time())}.csv"
        results_df.to_csv(results_filename, index=False)
        files.download(results_filename)

        print(f"送金結果を {results_filename} として保存しました。")


# 使用例
def main():
    print("ERC20トークン一括送金ツール")
    print("----------------------------")

    # ネットワーク設定
    networks = {
        "1": {"name": "イーサリアムメインネット", "url": "https://mainnet.infura.io/v3/YOUR_INFURA_PROJECT_ID"},
        "5": {"name": "Goerliテストネット", "url": "https://goerli.infura.io/v3/YOUR_INFURA_PROJECT_ID"},
        "11155111": {"name": "Sepoliaテストネット", "url": "https://sepolia.infura.io/v3/YOUR_INFURA_PROJECT_ID"},
        "137": {"name": "Polygonメインネット", "url": "https://polygon-rpc.com"},
        "80001": {"name": "Mumbaiテストネット", "url": "https://rpc-mumbai.maticvigil.com"},
        "56": {"name": "BSCメインネット", "url": "https://bsc-dataseed.binance.org/"},
        "97": {"name": "BSCテストネット", "url": "https://data-seed-prebsc-1-s1.binance.org:8545/"},
        "43114": {"name": "Avalancheメインネット", "url": "https://api.avax.network/ext/bc/C/rpc"},
        "43113": {"name": "Avalancheテストネット", "url": "https://api.avax-test.network/ext/bc/C/rpc"}
    }

    # ネットワーク選択
    print("利用可能なネットワーク:")
    for chain_id, network in networks.items():
        print(f"{chain_id}: {network['name']}")

    # カスタムネットワークのオプション
    print("0: カスタムRPC URL")

    network_choice = input("ネットワークを選択してください (チェーンID): ")

    if network_choice == "0":
        provider_url = input("RPC URLを入力してください: ")
    else:
        provider_url = networks.get(network_choice, {}).get("url")
        if not provider_url:
            print("無効なネットワーク選択です。デフォルトのイーサリアムメインネットを使用します。")
            provider_url = networks["1"]["url"]

    # InfuraのプロジェクトIDが必要な場合
    if "YOUR_INFURA_PROJECT_ID" in provider_url:
        infura_project_id = input("InfuraプロジェクトIDを入力してください: ")
        provider_url = provider_url.replace("YOUR_INFURA_PROJECT_ID", infura_project_id)

    # トークンアドレスを入力
    token_address = input("ERC20トークンのコントラクトアドレスを入力してください: ")

    # プライベートキーを入力（セキュリティ注意）
    private_key = input("送金元のプライベートキーを入力してください（0xなし）: ")

    # 送金ツールの初期化
    transfer_tool = ERC20BulkTransfer(provider_url, token_address, private_key)

    # CSVファイルをアップロード
    recipients_df = transfer_tool.upload_csv()

    # 送金実行
    if recipients_df is not None:
        transfer_tool.send_tokens(recipients_df)


if __name__ == "__main__":
    main()